In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model
from keras import losses
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [6]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 100
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.KLD(y_true, y_pred) # KLD 왜 써야할까,,,
    return loss1 + loss2

In [13]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
#encoded = LSTM(128, return_sequences=True)(encoded)
encoded = LSTM(64, activation='relu')(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
#decoded = LSTM(256, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=custom_loss, optimizer=Adam(lr=0.001))
#lstm_autoencoder_500 = lstm_autoencoder

In [14]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [15]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/100
2352/2352 [==============================] - 23s 10ms/step - loss: 21.7425
Epoch 2/100
2352/2352 [==============================] - 21s 9ms/step - loss: 13.3673
Epoch 3/100
2352/2352 [==============================] - 21s 9ms/step - loss: 11.9341
Epoch 4/100
2352/2352 [==============================] - 21s 9ms/step - loss: 10.8175
Epoch 5/100
2352/2352 [==============================] - 21s 9ms/step - loss: 9.6804
Epoch 6/100
2352/2352 [==============================] - 21s 9ms/step - loss: 8.3871
Epoch 7/100
2352/2352 [==============================] - 21s 9ms/step - loss: 7.6728
Epoch 8/100
2352/2352 [==============================] - 21s 9ms/step - loss: 6.9347
Epoch 9/100
2352/2352 [==============================] - 21s 9ms/step - loss: 5.8199
Epoch 10/100
2352/2352 [==============================] - 21s 9ms/step - loss: 4.9524
Epoch 11/100
2352/2352 [==============================] - 21s 9ms/step - loss: 4.4042
Epoch 12/100
2352/2352 [==============================] - 

2352/2352 [==============================] - 19s 8ms/step - loss: 0.2727
Epoch 97/100
2352/2352 [==============================] - 19s 8ms/step - loss: 0.3385
Epoch 98/100
2352/2352 [==============================] - 19s 8ms/step - loss: 0.3751
Epoch 99/100
2352/2352 [==============================] - 19s 8ms/step - loss: 0.3059
Epoch 100/100
2352/2352 [==============================] - 19s 8ms/step - loss: 0.2247


In [33]:
# model save
#lstm_autoencoder.save('model/lstm_autoencoder_custom_loss_bfs.h5')

In [ ]:
model_json = lstm_autoencoder.to_json()
with open('model/lstm_autoencoder_bfs.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model/weights_lstm_autoencoder_bfs.h5')

In [47]:
model = model_from_json(open('model/lstm_autoencoder_custom_loss_bfs.json').read())
model.load_weights('model/weights_lstm_autoencoder_custom_loss_bfs.h5')

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_33 (LSTM)                  (None, None, 128)    67584       input_7[0][0]                    
__________________________________________________________________________________________________
lstm_34 (LSTM)                  (None, 64)           49408       lstm_33[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (None, None, 64)     0           lstm_34[0][0]                    
                                                                 input_7[0][0]             

In [16]:
loaded_model = lstm_autoencoder

In [17]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

2.5025875809588274


In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [20]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [21]:
latent_vector = np.array(latent_vector)